In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

from datetime import datetime
from datetime import timedelta
# !pip install holidays
import holidays

In [2]:
os.chdir("/content/drive/MyDrive/3. Grad School/LG Aimers")
os.listdir()

['DATA',
 'baseline_submission.csv',
 'Models',
 '0. Baseline.ipynb',
 'baseline_submission_lstm.csv',
 'baseline_submission_hybrid.csv',
 '1. Trials.ipynb']

In [3]:
data = pd.read_csv("DATA/train/train.csv")
data.head()

,영업일자,영업장명_메뉴명,매출수량
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0


In [4]:
data.sort_values(by = "매출수량", ascending = False).head(10)

,영업일자,영업장명_메뉴명,매출수량
89753,2024-01-13,포레스트릿_꼬치어묵,1372
89767,2024-01-27,포레스트릿_꼬치어묵,1329
89760,2024-01-20,포레스트릿_꼬치어묵,1235
90299,2024-01-27,포레스트릿_떡볶이,1200
90285,2024-01-13,포레스트릿_떡볶이,1174
89410,2023-02-04,포레스트릿_꼬치어묵,1165
89398,2023-01-23,포레스트릿_꼬치어묵,1130
89746,2024-01-06,포레스트릿_꼬치어묵,1114
89403,2023-01-28,포레스트릿_꼬치어묵,1107
89733,2023-12-24,포레스트릿_꼬치어묵,1094


#### 파생변수 생성

In [5]:
# 월(month) -> 계절 매핑 딕셔너리
month_to_season = {
    1: "Winter", 2: "Winter", 12: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"
}

In [ ]:
kor_holidays2 = holidays.KR(years = [2025])
kor_holidays2

{datetime.date(2025, 1, 1): "New Year's Day", datetime.date(2025, 1, 29): 'Korean New Year', datetime.date(2025, 1, 28): 'The day preceding Korean New Year', datetime.date(2025, 1, 30): 'The second day of Korean New Year', datetime.date(2025, 3, 1): 'Independence Movement Day', datetime.date(2025, 5, 5): "Buddha's Birthday; Children's Day", datetime.date(2025, 6, 6): 'Memorial Day', datetime.date(2025, 8, 15): 'Liberation Day', datetime.date(2025, 10, 3): 'National Foundation Day', datetime.date(2025, 10, 9): 'Hangul Day', datetime.date(2025, 10, 6): 'Chuseok', datetime.date(2025, 10, 5): 'The day preceding Chuseok', datetime.date(2025, 10, 7): 'The second day of Chuseok', datetime.date(2025, 12, 25): 'Christmas Day', datetime.date(2025, 3, 3): 'Alternative holiday for Independence Movement Day', datetime.date(2025, 5, 6): "Alternative holiday for Buddha's Birthday; Alternative holiday for Children's Day", datetime.date(2025, 10, 8): 'Alternative holiday for Chuseok', datetime.date(202

대선 추가

In [6]:
class Make_Variables():
        def __init__(self, data = None, date = None, predict = 7):
            self.data = data
            self.date = date
            self.predict = predict

        # train, test의 입력 데이터
        def make_variables(self, data):
            # 영업일자 -> datetime
            data['영업일자'] = pd.to_datetime(data['영업일자'])

            # 연, 월, 일, 요일 분리
            data['year'] = data['영업일자'].dt.year
            data['month'] = data['영업일자'].dt.month
            data['day'] = data['영업일자'].dt.day
            data['weekday'] = data['영업일자'].dt.day_name()

            # 계절 변수 생성
            data['season'] = data['month'].map(month_to_season)

            # 공휴일 확인
            kor_holidays = holidays.KR(years = [2023, 2024, 2025])
            data['is_holiday'] = data['영업일자'].dt.date.isin(kor_holidays)
            data['holiday_name'] = data['영업일자'].dt.date.map(kor_holidays)

            # 샌드위치 데이
            check_sandwich = data['is_holiday'].astype(int).values
            prev, next = np.roll(check_sandwich, 1), np.roll(check_sandwich, -1)
            sandwich = (prev == 1) & (check_sandwich == 0) & (next == 1)
            data['is_sandwich'] = sandwich.astype(int)

            # 첫날
            first = data['영업일자'].min()
            prev_date, next_date = first - timedelta(days = 1), first + timedelta(days = 1)
            if (prev_date.date() in kor_holidays) and (next_date.date() in kor_holidays):
                data.loc[data['영업일자'] == first, 'is_sandwich'] = True

            # 마지막 날
            last = data['영업일자'].max()
            prev_date, next_date = last - timedelta(days = 1), last + timedelta(days = 1)
            if (prev_date.date() in kor_holidays) and (next_date.date() in kor_holidays):
                data.loc[data['영업일자'] == last, 'is_sandwich'] = True

            # 샌드위치 포함한 공휴일
            data['is_holiday_sandwich'] = data['is_holiday'] | data['is_sandwich']

            # 영업장명, 메뉴명 분리
            if '영업장명_메뉴명' in data.columns:
                data[['영업장명', '메뉴명']] = data['영업장명_메뉴명'].str.split('_', expand = True)

            return data

        # 예측하고자 하는 날들
        def make_variables_for_testing(self, date, predict):
            """date : 최종 날짜 (입력 7일 중 가장 마지막)"""
            future_dates = [date + timedelta(days = i + 1) for i in range(predict)]

            future_df = pd.DataFrame({'영업일자' : future_dates})

            # 파생변수 생성
            future_df = self.make_variables(future_df)

            return future_df

In [7]:
# 그냥 전부 만들면 돼
mv = Make_Variables()
data = mv.make_variables(data)

In [ ]:
cols = ["month", "weekday", "season", "is_holiday", "is_holiday_sandwich" ] # x 변수들

In [ ]:
print("기존 공휴일 : ", data['is_holiday'].sum().item())
print("샌드위치 : ", data['is_sandwich'].sum().item())
print("샌드위치 포함 공휴일 : ", data['is_holiday_sandwich'].sum().item())

기존 공휴일 :  5597
샌드위치 :  386
샌드위치 포함 공휴일 :  5983


In [9]:
data.head(3)

,영업일자,영업장명_메뉴명,매출수량,year,month,day,weekday,season,is_holiday,holiday_name,is_sandwich,is_holiday_sandwich,영업장명,메뉴명
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,2023,1,1,Sunday,Winter,True,New Year's Day,0,True,느티나무 셀프BBQ,1인 수저세트
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,2023,1,2,Monday,Winter,False,NaN,0,False,느티나무 셀프BBQ,1인 수저세트
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,2023,1,3,Tuesday,Winter,False,NaN,0,False,느티나무 셀프BBQ,1인 수저세트


In [ ]:
# 18
data.query("('2023-09-12' <= 영업일자 <= '2023-09-20') & (영업장명_메뉴명 == '느티나무 셀프BBQ_스프라이트 (단체)')")

In [ ]:
data.loc[(data['영업일자'] == '2023-09-19') & (data['영업장명_메뉴명'].str.startswith('느티나무'))]

In [ ]:
# 18
data.query("('2023-05-10' <= 영업일자 <= '2023-05-17') & (영업장명_메뉴명 == '연회장_Cass Beer')")

In [ ]:
# 26
data.query("('2024-04-16' <= 영업일자 <= '2024-04-23') & (영업장명_메뉴명 == '연회장_Cass Beer')")

In [ ]:
# 80
data.query("('2023-07-11' <= 영업일자 <= '2023-07-18') & (영업장명_메뉴명 == '카페테리아_단체식 18000(신)')")

In [ ]:
data[data['매출수량'] < 0]

큰 수만 처리 -> 18, 26, 80

In [ ]:
# 첫 번째 경우는 단순 오류로 판단해 drop, 나머지는 전날의 값을 수정
data_original = data.copy()

negative = data[data['매출수량'] < 0]

In [ ]:
for idx, row in negative.iterrows():
    num = row['매출수량']
    if num < -10:
        date = row['영업일자']
        menu = row['영업장명_메뉴명']
        prev_date = pd.to_datetime(date) - pd.Timedelta(days = 1)
        prev_row = data[(data['영업일자'] == prev_date) & (data['영업장명_메뉴명'] == menu)]

        if prev_row.iloc[0]["매출수량"] >= abs(num):
            data.loc[prev_row.index[0], '매출수량'] += num

In [ ]:
# 전부 0으로
data.loc[data['매출수량'] < 0, '매출수량'] = 0

#### Validation Set

In [ ]:
stores = data['영업장명'].unique()

def make_validation(data):
    dates = sorted(data['영업일자'].unique())
    n = int(len(dates) * 0.2)
    val_dates = dates[-n : ]
    val_mask = data['영업일자'].isin(val_dates)
    train = data[~val_mask]
    validation = data[val_mask]
    return train, validation

In [ ]:
train, validation = make_validation(data)
print(f"Train : {train['영업일자'].min()} ~ {train['영업일자'].max()}")
print(f"Validation : {validation['영업일자'].min()} ~ {validation['영업일자'].max()}")

Train : 2023-01-01 00:00:00 ~ 2024-03-01 00:00:00
Validation : 2024-03-02 00:00:00 ~ 2024-06-15 00:00:00


#### 0 여부 분류

In [ ]:
from xgboost import XGBClassifier
! pip install category_encoders
from category_encoders import TargetEncoder
from collections import defaultdict
import pickle
import joblib
# from sklearn.metrics import classification_report

In [ ]:
# 매출수량 변환
data_zero = data.copy()
data_zero['매출_여부'] = data_zero['매출수량'].apply(lambda x:1 if x > 0 else 0)

- 방법 1.
    - `영업장명_메뉴명`으로 groupby 후, 각 메뉴별로 XGBoost 모델
- 방법 2.
    - `영업장명_메뉴명`을 x 변수로 넣어서 모델 학습..
- 방법 3.
    - 판매량 많은 거나 편차 큰 메뉴는 개별 모델, 나머지는 그냥 하나로 통합..? <- 왕창 복잡할듯

In [ ]:
### 방법 1로..
# 모델 담을 곳
models = {}

# 사용 변수 설정
cols = ["month", "weekday", "season", "is_holiday", "is_holiday_sandwich" ] # x 변수들

# train, validation 설정 ########### 추후에 cross validation 추가 예정
train_cls, validation_cls = make_validation(data_zero)

# 메뉴별로 모델 fit
def fit_model_by_menu(train_cls, validation_cls, cols):
    for menu, group_df in train_cls.groupby("영업장명_메뉴명"):

        # 수량 전부 0이거나 0 아닌 날 없으면 학습 불가
        if group_df['매출_여부'].nunique() < 2:
            print(f"{menu} 학습 불가")
            continue

        # 범주형 변수 처리
        target_encoder = TargetEncoder()
        group_df[cols] = target_encoder.fit_transform(group_df[cols], group_df['매출_여부'])

        # x, y 분리
        x_train = group_df[cols]
        y_train = group_df["매출_여부"]
        val_group = validation_cls[validation_cls["영업장명_메뉴명"] == menu]
        x_test = target_encoder.transform(val_group[cols])
        y_test = val_group["매출_여부"]

        # 모델 설정
        xgb_model = XGBClassifier(random_state = 1471)

        # 모델 학습
        xgb_model.fit(x_train, y_train)
        # y_pred = xgb_model.predict(x_test)

        models[menu] = {
            "model" : xgb_model,
            "encoder" : target_encoder
        }

        print(f"모델 학습 완료 : {menu}")
    return models

In [ ]:
models = fit_model_by_menu(train_cls, validation_cls, cols)

모델 학습 완료 : 느티나무 셀프BBQ_1인 수저세트
모델 학습 완료 : 느티나무 셀프BBQ_BBQ55(단체)
모델 학습 완료 : 느티나무 셀프BBQ_대여료 30,000원
모델 학습 완료 : 느티나무 셀프BBQ_대여료 60,000원
모델 학습 완료 : 느티나무 셀프BBQ_대여료 90,000원
모델 학습 완료 : 느티나무 셀프BBQ_본삼겹 (단품,실내)
모델 학습 완료 : 느티나무 셀프BBQ_스프라이트 (단체)
모델 학습 완료 : 느티나무 셀프BBQ_신라면
모델 학습 완료 : 느티나무 셀프BBQ_쌈야채세트
모델 학습 완료 : 느티나무 셀프BBQ_쌈장
모델 학습 완료 : 느티나무 셀프BBQ_육개장 사발면
모델 학습 완료 : 느티나무 셀프BBQ_일회용 소주컵
모델 학습 완료 : 느티나무 셀프BBQ_일회용 종이컵
모델 학습 완료 : 느티나무 셀프BBQ_잔디그늘집 대여료 (12인석)
모델 학습 완료 : 느티나무 셀프BBQ_잔디그늘집 대여료 (6인석)
모델 학습 완료 : 느티나무 셀프BBQ_잔디그늘집 의자 추가
모델 학습 완료 : 느티나무 셀프BBQ_참이슬 (단체)
모델 학습 완료 : 느티나무 셀프BBQ_친환경 접시 14cm
모델 학습 완료 : 느티나무 셀프BBQ_친환경 접시 23cm
모델 학습 완료 : 느티나무 셀프BBQ_카스 병(단체)
모델 학습 완료 : 느티나무 셀프BBQ_콜라 (단체)
모델 학습 완료 : 느티나무 셀프BBQ_햇반
모델 학습 완료 : 느티나무 셀프BBQ_허브솔트
모델 학습 완료 : 담하_(단체) 공깃밥
모델 학습 완료 : 담하_(단체) 생목살 김치전골 2.0
모델 학습 완료 : 담하_(단체) 은이버섯 갈비탕
모델 학습 완료 : 담하_(단체) 한우 우거지 국밥
모델 학습 완료 : 담하_(단체) 황태해장국 3/27까지
모델 학습 완료 : 담하_(정식) 된장찌개
모델 학습 완료 : 담하_(정식) 물냉면 
모델 학습 완료 : 담하_(정식) 비빔냉면
모델 학습 완료 : 담하_(후식) 된장찌개
모델 학습 완료 : 담하_(후식) 물냉면
모델 학습 완료 : 담하_

전부 0이나 1로 예측해버리는 메뉴들이 있네.......

문제가 있긴 하지만.. 일단은 진행해보자

In [ ]:
# 일단 저장해두기..
import joblib
joblib.dump(models, '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/cls_models.pkl')

['/content/drive/MyDrive/3. Grad School/LG Aimers/Models/cls_models.pkl']

In [ ]:
models_class = joblib.load('/content/drive/MyDrive/3. Grad School/LG Aimers/Models/cls_models.pkl')

In [ ]:
def predict_class(test_df, trained_models, test_prefix : str, cols : list, predict = 7):
    """
    Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
    Output : [영업일자, 영업장명_메뉴명, 매출여부] DataFrame
    """
    results = []

    for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
        store_menu = store_menu_tup[0]
        # 훈련된 모델에 메뉴가 있는 경우만 진행
        if store_menu not in trained_models:
            continue

        # 모델 불러오기
        model = trained_models[store_menu]["model"]
        encoder = trained_models[store_menu]["encoder"]

        # 변수 추가하기
        mv = Make_Variables()
        last_date = pd.to_datetime(store_test['영업일자'].max())
        store_test = mv.make_variables_for_testing(last_date, predict = 7)

        store_test_enc = encoder.transform(store_test[cols])
        store_test_enc = store_test_enc[cols]

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

        for d, val in zip(pred_dates, model.predict(store_test_enc)):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출여부': val
            })

    return pd.DataFrame(results)

#### 매출 예측 (회귀)

매출 > 0 인 데이터만 가지고 학습 진행

In [ ]:
from xgboost import XGBRegressor
! pip install category_encoders
from category_encoders import TargetEncoder
from collections import defaultdict
from sklearn.metrics import mean_squared_error, r2_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
# 매출 양수 필터링
data_notzero = data[data['매출수량'] > 0]

# 모델 담을 곳
models = {}

# 사용 변수 설정
cols = ["month", "weekday", "season", "is_holiday", "is_holiday_sandwich" ] # x 변수들

# train, validation 설정 ########### 추후에 cross validation 추가 예정
train_reg, validation_reg = make_validation(data_notzero)

# 메뉴별로 모델 fit
def fit_model_by_menu(train_reg, validation_reg, cols):
    for menu, group_df in train_reg.groupby("영업장명_메뉴명"):

        # 데이터 수 적으면 학습 불가
        if len(group_df) < 10:
            print(f"{menu} 학습 불가")
            continue

        # 범주형 변수 처리
        target_encoder = TargetEncoder()
        group_df[cols] = target_encoder.fit_transform(group_df[cols], group_df['매출수량'])

        # x, y 분리
        x_train = group_df[cols]
        y_train = group_df["매출수량"]
        val_group = validation_reg[validation_reg["영업장명_메뉴명"] == menu]
        x_test = target_encoder.transform(val_group[cols])
        y_test = val_group["매출수량"]

        # 모델 설정
        xgb_model = XGBRegressor(random_state = 1471)

        # 모델 학습
        xgb_model.fit(x_train, y_train)
        # y_pred = xgb_model.predict(x_test)

        models[menu] = {
            "model" : xgb_model,
            "encoder" : target_encoder
        }

        print(f"모델 학습 완료 : {menu}")
    return models

In [ ]:
models = fit_model_by_menu(train_reg, validation_reg, cols)

In [ ]:
import joblib
joblib.dump(models, '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/reg_models.pkl')

['/content/drive/MyDrive/3. Grad School/LG Aimers/Models/reg_models.pkl']

In [ ]:
models_reg = joblib.load('/content/drive/MyDrive/3. Grad School/LG Aimers/Models/reg_models.pkl')

우와 성능,,,

In [ ]:
def predict_reg(test_df, trained_models, test_prefix : str, cols : list, predict = 7):
    """
    Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
    Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
    """
    results = []

    for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
        store_menu = store_menu_tup[0]
        # 훈련된 모델에 메뉴가 있는 경우만 진행
        if store_menu not in trained_models:
            continue

        # 모델 불러오기
        model = trained_models[store_menu]["model"]
        encoder = trained_models[store_menu]["encoder"]

        # 변수 추가하기
        mv = Make_Variables()
        last_date = pd.to_datetime(store_test['영업일자'].max())
        store_test = mv.make_variables_for_testing(last_date, predict = 7)

        store_test_enc = encoder.transform(store_test[cols])
        store_test_enc = store_test_enc[cols]

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

        for d, val in zip(pred_dates, model.predict(store_test_enc)):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량(reg)': val
            })

    return pd.DataFrame(results)

#### 매출 예측 (시계열)

시계열 특성 살리려면 매출 = 0인 데이터도 포함해야지..

-> 전체 데이터로 진행

- 추후) RevIN 사용해보기

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import random
import glob

import torch
import torch.nn as nn
from tqdm import tqdm

In [ ]:
### Random Seed & Parameters
def set_seed(seed = 1471):
    random.seed(seed) # 일반 seed
    np.random.seed(seed) # numpy 난수 고정
    torch.manual_seed(seed) # CPU 난수 고정
    os.environ["PYTHONHASHSEED"] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(1471)

In [ ]:
lookback, predict, batch_size, epochs = 28, 7, 16, 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 전처리 여기서 다 하기
class Preprocess_LSTM():
    def __init__(self, data = None, cols = None, scaler = None):
        self.data = data
        self.cols = cols
        self.scaler = scaler

    # 범주형 변수는 Label Encoding
    def label_encoding_lstm(self, data, cols):
        for col in cols:
            if data[col].dtype == 'object' or data[col].dtype.name == 'bool' or data[col].dtype.name == 'category':
                le = LabelEncoder()
                data[col] = le.fit_transform(data[col])
        return data

    # 매출수량은 MinMaxScaling
    def minmax_scaling_lstm(self, data, scaler):
        if scaler is None:
            scaler = MinMaxScaler()
        data['매출수량'] = scaler.fit_transform(data[['매출수량']])

        return data, scaler

In [ ]:
class MultiOutputLSTM(nn.Module):
        def __init__(self, input_dim = 1, hidden_dim = 64, num_layers = 2, output_dim = 7):
            """ 7개 값 예측 (PREDICT 만큼의 날짜의 값을 예측하고자 함)"""
            super(MultiOutputLSTM, self).__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first = True)
            self.fc = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            out, _ = self.lstm(x)
            return self.fc(out[:, -1, :]) # 마지막 시점 출력만 선택해서 fc에 넣음 -> (batch * output_dim)

In [ ]:
def train_lstm(train_df, scaler, cols):
    """
    영업장, 메뉴별로 LSTM 모델 훈련, 각각을 trained_models에 저장
    1. 전체 데이터를 '영업장명_메뉴명'으로 나누고 -> 각 데이터를 정규화, LSTM 학습
    """
    trained_models = {}

    # store_menu : 영업장명_메뉴명 / group : 나머지 데이터
    for store_menu, group in tqdm(train_df.groupby(["영업장명_메뉴명"]), desc = "Training LSTM"):

        # 날짜 순으로 정렬해서 데이터가 너무 적으면 -> 학습 생략
        store_train = group.sort_values("영업일자").copy()
        if len(store_train) < lookback + predict:
            continue

        # 예측에 사용할 변수들 : cols
        train_vals = store_train[cols].values
        target_vals = store_train["매출수량"].values

        # lookback : 과거 데이터 며칠 쓸지
        # predict : 미래 데이터 며칠 예측할 건지
        # 입력과 출력 (x_train, y_train) 생성
        x_train, y_train = [], []
        for i in range(len(train_vals) - lookback - predict + 1):
            x_train.append(train_vals[i : i + lookback])
            y_train.append(target_vals[i + lookback : i + lookback + predict])

        # 텐서 변환
        x_train = torch.tensor(x_train).float().to(device)
        y_train = torch.tensor(y_train).float().to(device)

        # 모델 초기화 (영업장_메뉴별로 다른 모델)
        model = MultiOutputLSTM(input_dim = len(cols), output_dim = predict).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
        criterion = nn.MSELoss()

        # 학습 모드로 설정
        model.train()

        # epochs 만큼 훈련
        for epoch in range(epochs):
            # idx : 랜덤하게 섞인 index들
            idx = torch.randperm(len(x_train))
            for i in range(0, len(x_train), batch_size):
                batch_idx = idx[i:i+batch_size] # 배치 개수만큼 끊어서
                x_batch, y_batch = x_train[batch_idx], y_train[batch_idx] # 배치 데이터 할당
                output = model(x_batch) # 모델 태워서
                loss = criterion(output, y_batch) # 평가하고
                optimizer.zero_grad() # 역전파를 위한 초기화
                loss.backward() # 역전파
                optimizer.step() # 최적화

        # 모델 저장
        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-lookback:]  # (28, 1)
        }

    return trained_models

In [ ]:
# 데이터 준비
cols = ["month", "weekday", "season", "is_holiday", "is_holiday_sandwich" ]
features = cols + ["매출수량", "영업일자", "영업장명_메뉴명"]
dataset_lstm = data[features]

pp = Preprocess_LSTM()
dataset_lstm = pp.label_encoding_lstm(dataset_lstm, cols)
dataset_lstm, scaler = pp.minmax_scaling_lstm(dataset_lstm, scaler = None)
trainset_lstm, validationset_lstm = make_validation(dataset_lstm)

In [ ]:
# 학습
trained_lstm = train_lstm(trainset_lstm, scaler, cols)

Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]/tmp/ipython-input-3610431613.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  x_train = torch.tensor(x_train).float().to(device)
Training LSTM: 100%|██████████| 193/193 [03:45<00:00,  1.17s/it]


In [ ]:
# 모델 저장
import joblib
joblib.dump(trained_lstm, '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/lstm_models.pkl')

torch.save(trained_lstm, '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/lstm_models.pt')

In [ ]:
models_lstm = joblib.load('/content/drive/MyDrive/3. Grad School/LG Aimers/Models/lstm_models.pkl')
models_lstm

{('느티나무 셀프BBQ_1인 수저세트',): {'model': MultiOutputLSTM(
    (lstm): LSTM(5, 64, num_layers=2, batch_first=True)
    (fc): Linear(in_features=64, out_features=7, bias=True)
  ),
  'scaler': MinMaxScaler(),
  'last_sequence': array([[2, 2, 3, 0, 0],
         [2, 3, 3, 0, 0],
         [2, 1, 3, 0, 0],
         [2, 5, 3, 0, 0],
         [2, 6, 3, 0, 0],
         [2, 4, 3, 0, 0],
         [2, 0, 3, 1, 1],
         [2, 2, 3, 1, 1],
         [2, 3, 3, 1, 1],
         [2, 1, 3, 1, 1],
         [2, 5, 3, 0, 0],
         [2, 6, 3, 0, 0],
         [2, 4, 3, 0, 0],
         [2, 0, 3, 0, 0],
         [2, 2, 3, 0, 0],
         [2, 3, 3, 0, 0],
         [2, 1, 3, 0, 0],
         [2, 5, 3, 0, 0],
         [2, 6, 3, 0, 0],
         [2, 4, 3, 0, 0],
         [2, 0, 3, 0, 0],
         [2, 2, 3, 0, 0],
         [2, 3, 3, 0, 0],
         [2, 1, 3, 0, 0],
         [2, 5, 3, 0, 0],
         [2, 6, 3, 0, 0],
         [2, 4, 3, 0, 0],
         [3, 0, 1, 1, 1]])},
 ('느티나무 셀프BBQ_BBQ55(단체)',): {'model': MultiOutputL

In [ ]:
def predict_lstm(test_df, trained_models, test_prefix : str, cols : list, predict : int):
    """
    Input : test_df - test data, trained_models - list(menu : { model}), cols - x 변수들
    Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
    """
    results = []

    # 매장, 메뉴별로 그룹화해서 예측
    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        # 훈련된 모델에 메뉴가 있는 경우만 진행
        if store_menu not in trained_models:
            continue

        # 모델, scaler 불러오기
        model = trained_models[store_menu]['model']
        scaler = trained_models[store_menu]['scaler']

        # LSTM 입력으로 활용할 최근 lookback 만큼의 데이터 가져오기
        mv = Make_Variables()
        store_test = mv.make_variables(store_test)
        store_test_sorted = store_test.sort_values('영업일자')

        features = cols + ["매출수량"]
        if len(store_test_sorted) < lookback:
            continue

        recent_df = store_test_sorted[features].iloc[-lookback:].copy()
        if len(recent_df) < lookback:
            continue # lookback 만큼의 데이터가 없으면 예측 안 하고 넘어가기

        ##### 요기서 변수 추가
        last_date = store_test_sorted['영업일자'].iloc[-1]
        future_df = mv.make_variables_for_testing(last_date, predict)
        future_df['매출수량'] = 0.0
        full_df = pd.concat([recent_df, future_df[features]], axis = 0)

        # 정규화
        pp = Preprocess_LSTM()
        full_df = pp.label_encoding_lstm(full_df, cols)
        full_df, _ = pp.minmax_scaling_lstm(full_df, scaler)
        # full_df['매출수량'] = scaler.transform(full_df[['매출수량']])
        x_input_vals = full_df[cols].values
        x_input = x_input_vals[:lookback]
        x_input = torch.tensor([x_input]).float().to(device)

        # 예측 수행
        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역정규화
        restored = []
        for i in range(predict):
            dummy = np.zeros((1, len(features)))
            dummy[0, features.index("매출수량")] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, features.index("매출수량")]
            restored.append(max(restored_val, 0)) # 음수 나오면 0으로 처리

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량(lstm)': val
            })

    return pd.DataFrame(results)

#### 예측하기

In [ ]:
import re
import glob
all_preds_class = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # 일단 분류 모델 넣고
    pred_class = predict_class(test_df, models_class, test_prefix, cols, predict = 7)
    all_preds_class.append(pred_class)

pred_class_df = pd.concat(all_preds_class, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
pred_class_df

,영업일자,영업장명_메뉴명,매출여부
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,1
1,TEST_00+2일,느티나무 셀프BBQ_1인 수저세트,1
2,TEST_00+3일,느티나무 셀프BBQ_1인 수저세트,0
3,TEST_00+4일,느티나무 셀프BBQ_1인 수저세트,1
4,TEST_00+5일,느티나무 셀프BBQ_1인 수저세트,1
...,...,...,...
13505,TEST_09+3일,화담숲카페_현미뻥스크림,1
13506,TEST_09+4일,화담숲카페_현미뻥스크림,1
13507,TEST_09+5일,화담숲카페_현미뻥스크림,1
13508,TEST_09+6일,화담숲카페_현미뻥스크림,1


In [ ]:
pred_class_df['매출여부'].sum()
# target encoding으로 바꾸니까 확 늘었다,,

np.int64(6190)

In [ ]:
import re
all_preds_class = []
all_preds_reg = []
all_preds_lstm = []


# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # 일단 분류 모델 넣고
    pred_class = predict_class(test_df, models_class, test_prefix, cols, predict = 7)
    all_preds_class.append(pred_class)

    # 1 나오면 회귀 모델 넣고
    pred_reg = predict_reg(test_df, models_reg, test_prefix, cols, predict = 7)
    all_preds_reg.append(pred_reg)

    # lstm 넣고
    pred_lstm = predict_lstm(test_df, trained_lstm, test_prefix, cols, predict = 7)
    all_preds_lstm.append(pred_lstm)

    # 합치기 (가중치.. 일단은 1.5 / 8.5 정도....)

df_class = pd.concat(all_preds_class, ignore_index=True)
df_reg   = pd.concat(all_preds_reg, ignore_index=True)
df_lstm  = pd.concat(all_preds_lstm, ignore_index=True)

full_pred_df = pd.merge(df_class, df_reg, on=['영업일자', '영업장명_메뉴명'], how='outer')

In [ ]:
full_pred_df = pd.merge(df_class, df_reg, on=['영업일자', '영업장명_메뉴명'], how='outer')
# df_lstm['영업장명_메뉴명'] = df_lstm['영업장명_메뉴명'].apply(lambda x : x[0])
full_pred_df = pd.merge(df_lstm_plz, full_pred_df, on=['영업일자', '영업장명_메뉴명'], how='outer')
full_pred_df

,영업일자,영업장명_메뉴명,매출수량,매출여부,매출수량(reg)
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,0.116629,1,7.601994
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),6.487981,0,63.987808
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",0.031307,1,11.399564
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",0.040198,1,3.598365
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",0.002678,1,1.248767
...,...,...,...,...,...
13505,TEST_09+7일,화담숲카페_메밀미숫가루,3.695694,1,53.999870
13506,TEST_09+7일,화담숲카페_아메리카노 HOT,7.859884,1,52.667622
13507,TEST_09+7일,화담숲카페_아메리카노 ICE,16.007752,1,130.994705
13508,TEST_09+7일,화담숲카페_카페라떼 ICE,0.993638,1,31.333733


In [ ]:
full_pred_df.rename(columns={'매출수량': '매출수량(lstm)'}, inplace=True)

In [ ]:
full_pred_df['매출수량'] = np.where(
    full_pred_df['매출여부'] == 1,
    full_pred_df['매출수량(reg)'] * 0.1 + full_pred_df['매출수량(lstm)'] * 0.9,
    full_pred_df['매출수량(lstm)']
)

full_pred_df.drop(columns=['매출여부', '매출수량(reg)', '매출수량(lstm)'], inplace=True)
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,0.865166
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),6.487981
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",1.168133
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",0.396015
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",0.127287
...,...,...,...
13505,TEST_09+7일,화담숲카페_메밀미숫가루,8.726112
13506,TEST_09+7일,화담숲카페_아메리카노 HOT,12.340658
13507,TEST_09+7일,화담숲카페_아메리카노 ICE,27.506448
13508,TEST_09+7일,화담숲카페_카페라떼 ICE,4.027647


In [ ]:
df_lstm_plz

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,0.116629
1,TEST_00+2일,느티나무 셀프BBQ_1인 수저세트,0.115101
2,TEST_00+3일,느티나무 셀프BBQ_1인 수저세트,0.081615
3,TEST_00+4일,느티나무 셀프BBQ_1인 수저세트,0.135826
4,TEST_00+5일,느티나무 셀프BBQ_1인 수저세트,0.094401
...,...,...,...
13505,TEST_09+3일,화담숲카페_현미뻥스크림,4.439127
13506,TEST_09+4일,화담숲카페_현미뻥스크림,4.978948
13507,TEST_09+5일,화담숲카페_현미뻥스크림,3.523658
13508,TEST_09+6일,화담숲카페_현미뻥스크림,3.715825


In [ ]:
df_lstm

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,"(느티나무 셀프BBQ_1인 수저세트,)",0.116629
1,TEST_00+2일,"(느티나무 셀프BBQ_1인 수저세트,)",0.115101
2,TEST_00+3일,"(느티나무 셀프BBQ_1인 수저세트,)",0.081615
3,TEST_00+4일,"(느티나무 셀프BBQ_1인 수저세트,)",0.135826
4,TEST_00+5일,"(느티나무 셀프BBQ_1인 수저세트,)",0.094401
...,...,...,...
13505,TEST_09+3일,"(화담숲카페_현미뻥스크림,)",4.439127
13506,TEST_09+4일,"(화담숲카페_현미뻥스크림,)",4.978948
13507,TEST_09+5일,"(화담숲카페_현미뻥스크림,)",3.523658
13508,TEST_09+6일,"(화담숲카페_현미뻥스크림,)",3.715825


In [ ]:
full_pred_df['영업장명_메뉴명'] = aaaaa

In [ ]:
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,0.0,느티나무 셀프BBQ_1인 수저세트,0.0
1,0.0,느티나무 셀프BBQ_1인 수저세트,0.0
2,0.0,느티나무 셀프BBQ_1인 수저세트,0.0
3,0.0,느티나무 셀프BBQ_1인 수저세트,0.0
4,0.0,느티나무 셀프BBQ_1인 수저세트,0.0
...,...,...,...
13505,0.0,화담숲카페_현미뻥스크림,0.0
13506,0.0,화담숲카페_현미뻥스크림,0.0
13507,0.0,화담숲카페_현미뻥스크림,0.0
13508,0.0,화담숲카페_현미뻥스크림,0.0


In [ ]:
df_lstm_plz = df_lstm.copy()
df_lstm_plz['영업장명_메뉴명'] = df_lstm['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량'].astype(float)
    ))

    final_df = sample_submission.copy()

    menu_cols = final_df.columns[1:]
    final_df[menu_cols] = final_df[menu_cols].astype(float)

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)


    return final_df

In [ ]:
sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_hybrid = convert_to_submission_format(full_pred_df, sample_submission)
final_hybrid.to_csv('baseline_submission_hybrid.csv', index=False, encoding='utf-8-sig')

In [ ]:
sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_lstm = convert_to_submission_format(df_lstm_plz, sample_submission)
final_lstm.to_csv('baseline_submission_lstm.csv', index=False, encoding='utf-8-sig')

In [ ]:
cols

['month', 'weekday', 'season', 'is_holiday', 'is_holiday_sandwich']